# Group Assignment 2 : Predictive Modelling of tip_amount 

## Objective 

The goal is to develop a predictive model that estimates the tip amount given to taxi drivers. Performance must be evaluated using Mean Absolute Error (MAE). 

## Importing libraries and dataset
Note that df_test represents the dataframe we have to use our model to predict tip_amount and X_test is a test set I defined for evaluating model performance. 

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import base, pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from feature_engine import encoding, imputation
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from typing import Any, Dict, Union
import copy
from sklearn.model_selection import KFold
pd.options.mode.copy_on_write = True 

In [134]:
file_path_1 = 'test.csv'  
df_test = pd.read_csv(file_path_1)

In [135]:
file_path = 'train.csv'  
df_train = pd.read_csv(file_path)
df_train = df_train.drop('weight', axis = 1)

## Data Preprocessing Pipeline: Summary

In this section, we've constructed data preprocessing pipeline that transforms, filters, and cleans the dataset. Breakdown: 

1. **Feature Engineering**: 

2. **Outlier Detection and Filtering**:
3. **Additional Transformations**:
   - Added weather information by merging external data.
   - Replaced infinite and NaN values.
   - Dropped irrelevant columns 

In [136]:
def length_time_hours(X): 
    X['length_time_hours'] = X['length_time'] / 3600
    return X.drop('length_time', axis = 1)

def trip_distance_km(X):
    X['trip_distance_km'] = X['trip_distance'] * 1.60934
    return X.drop('trip_distance', axis = 1)


def calculate_velocity(X):
    # Calculate velocity (trip_distance / length_time_hours) and assign it to a new column
    X['velocity'] = np.divide(X['trip_distance_km'], X['length_time_hours'])
    
    return X

def cost_per_mile(X): 
    # Calculate velocity (trip_distance / length_time_hours) and assign it to a new column
    X['cost_per_km'] = np.divide(X['fare_amount'], X['trip_distance_km'])
    
    return X  

def cost_per_hour(X): 
    # Calculate velocity (trip_distance / length_time_hours) and assign it to a new column
    X['cost_per_hour'] = np.divide(X['fare_amount'], X['length_time_hours'] * 60)
    
    return X  

def cost_per_pair(X):
    
    # Calculate mean fare_amount for each pair
    pair_means = X.groupby('pair')['fare_amount'].mean()
    
    # Calculate fare_amount percentage variation from the mean of the pair group
    X['fare_amount_pair_variation'] = X.apply(lambda row: ((row['fare_amount'] - pair_means[row['pair']]) / pair_means[row['pair']]) * 100, axis=1)
    
    return X
    
def fare_per_passenger(X): 
    # Compute passenger fare 
    X['fare_per_passenger'] = np.divide(X['fare_amount'], X['passenger_count'])

    return X

def estimated_fare(X): 
    def calculate_fare(row):
        # Base fee and kilometer price
        if row['pickup_hour'] >= 20 or (row['pickup_hour'] < 6 and row['pickup_wday'] in [5, 6]):
            base_fee = 3.50  # Adjusted from $4.00 to $3.50
        else:
            base_fee = 2.50  # Adjusted from $3.00 to $2.50
        kilometer_price = 1.75  # Adjusted from $2.18 to $1.75
        
        # Calculate distance-based fare
        distance_fare = kilometer_price * row['trip_distance_km']
        
        # Waiting time fare
        waiting_time_fare = 0.00  # Assume no waiting time information available
        
        # Apply surcharges
        if row['pickup_hour'] >= 16 and row['pickup_hour'] < 20:
            distance_fare += 1.00  # $1 surcharge for trips between 4pm to 8pm on weekdays
        if row['dropoff_BoroCode'] in [1, 4, 5, 6, 9, 10, 12, 13]:  # Check if dropoff is within NYC
            distance_fare += 0.50  # MTA State Surcharge
        
        # Total fare
        total_fare = base_fee + distance_fare + waiting_time_fare
        
        return total_fare
    
    # Apply fare calculation function to each row and create a new column 'estimated_fare'
    X['estimated_fare'] = X.apply(calculate_fare, axis=1)
    return X


def fare_differences(X): 
    X['diff_fares'] = X['fare_amount'] - X['estimated_fare']
    return X

def drop_columns(X): 
    X = copy.deepcopy(X)
    #X.drop(['pickup_month', 'dropoff_BoroCode', 'pickup_BoroCode', 
    #           'pickup_NTACode', 'dropoff_NTACode', 'pickup_longitude', 'pickup_latitude', 
    #           'dropoff_longitude', 'dropoff_latitude', 'pickup_doy', 'pickup_week', 'velocity'], inplace = True, axis = 1)
    
    return X.drop(['pickup_month', 'dropoff_BoroCode', 'pickup_BoroCode', 
               'pickup_week', 'pickup_NTACode', 'dropoff_NTACode', 'estimated_fare', 'pair', 'pickup_datetime', 
               'pickup_dom', 'pickup_year', 'time', 'pickup_wday'], axis = 1)

import calendar

def add_datetime_column(X):
    def is_leap_year(year):
        return calendar.isleap(year)
    
    def day_of_year_to_day_of_month(year, day_of_year):
        if is_leap_year(year):
            days_in_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        else:
            days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
        for month in range(12):
            if sum(days_in_month[:month+1]) >= day_of_year:
                return day_of_year - sum(days_in_month[:month])
                
    
    # add day of month
    X.loc[:, 'pickup_dom'] = X['pickup_doy'].apply(lambda x: day_of_year_to_day_of_month(2015, x))
    # add year
    X['pickup_year'] = 2015
    # create the datetime of pick up column
    X['pickup_datetime'] = pd.to_datetime({'year': X['pickup_year'],
                                                   'month': X['pickup_month'],
                                                   'day': X['pickup_dom'],
                                                   'hour': X['pickup_hour']
                                                  })
    return X



def add_weather_info(df):
    # open weather dataframe
    df_weather = pd.read_csv('new_york_weather.csv', skiprows=2)
    df_weather['time'] = pd.to_datetime(df_weather['time'])
    df_temp = df.copy()
    
    # Store the original index
    df_temp['original_index'] = df_temp.index
    
    # Sort required for merging
    df_temp = df_temp.sort_values(by='pickup_datetime')
    
    # Merge dataframes on a temporary dataframe
    df_temp = pd.merge_asof(df_temp, df_weather, left_on='pickup_datetime', right_on='time')
    
    # Join back to original indices to maintain order
    df_return = pd.merge(df, df_temp, how='left', left_index=True, right_on='original_index')
    df_return = df_return.set_index('original_index')  # Set original index back
    
    # Remove duplicate columns and rename columns
    duplicate_columns = [col for col in df_return.columns if '_x' in col or '_y' in col]
    for col in duplicate_columns:
        new_col_name = col[:-2]  # Remove '_x' or '_y'
        df_return.rename(columns={col: new_col_name}, inplace=True)
    
    # Remove exact duplicate columns
    df_return = df_return.loc[:,~df_return.columns.duplicated()]
    
    return df_return

def replace_inf_nan(X):
    return X.replace([np.inf, -np.inf], np.nan)


In [157]:
def calculate_upperbound(X, feature):
    feature_data = X[feature]
    
    Q1 = feature_data.quantile(0.25)
    Q3 = feature_data.quantile(0.75)
    
    IQR = Q3 - Q1
    
    upper_bound = Q3 + 1.5 * IQR
    
    return upper_bound

def length_time_hours(X): 
    X['length_time_hours'] = X['length_time'] / 3600
    return X.drop('length_time', axis = 1)

def trip_distance_km(X):
    X['trip_distance_km'] = X['trip_distance'] * 1.60934
    return X.drop('trip_distance', axis = 1)

def calculate_velocity(X):
    X['velocity'] = np.divide(X['trip_distance_km'], X['length_time_hours'])
    return X

def filter_trip_distance_km(X): 
    upperbound_distance = calculate_upperbound(X, 'trip_distance_km')
    upperbound_length_time = calculate_upperbound(X, 'length_time_hours')

    # Case 1: Large distance, regular time
    X_outliers = X[(X['trip_distance_km'] > upperbound_distance) & (X['length_time_hours'] <= upperbound_length_time)]
    X_outliers = X_outliers[X_outliers['velocity'] > 100]
    X.loc[X_outliers.index, 'trip_distance_km'] = np.NaN

    # Case 2: Irregular distance, irregular time
    X_outliers = X[(X['trip_distance_km'] > upperbound_distance) & (X['length_time_hours'] > upperbound_length_time)]
    X_outliers = X_outliers[(X_outliers['velocity'] > 100) | (X_outliers['length_time_hours'] >= 3)]
    X.loc[X_outliers.index, 'trip_distance_km'] = np.NaN

    # Case 3: Zero cases
    X_outliers = X[(X['length_time_hours'] == 0) | (X['trip_distance_km'] == 0)]
    X.loc[X_outliers.index, 'trip_distance_km'] = np.NaN

    # Case 4: Regular distance, regular time
    X_outliers = X[(X['trip_distance_km'] <= upperbound_distance) & (X['length_time_hours'] <= upperbound_length_time)]
    X_outliers = X_outliers[X_outliers['velocity'] > 100]
    X.loc[X_outliers.index, 'trip_distance_km'] = np.NaN

    return X

def filter_length_time(X): 
    upperbound_distance = calculate_upperbound(X, 'trip_distance_km')
    upperbound_length_time = calculate_upperbound(X, 'length_time_hours')

    # Case 1: Regular distance, large time
    X_outliers = X[(X['trip_distance_km'] <= upperbound_distance) & (X['length_time_hours'] > upperbound_length_time)]
    X_outliers = X_outliers[(X_outliers['velocity'] > 100) | (X_outliers['length_time_hours'] >= 3)]
    X.loc[X_outliers.index, 'length_time_hours'] = np.NaN

    # Case 2: Zero cases
    X_outliers = X[(X['length_time_hours'] == 0) | (X['trip_distance_km'] == 0)]
    X.loc[X_outliers.index, 'length_time_hours'] = np.NaN

    # Case 3: Irregular distance, irregular time
    X_outliers = X[(X['trip_distance_km'] > upperbound_distance) & (X['length_time_hours'] > upperbound_length_time)]
    X_outliers = X_outliers[(X_outliers['velocity'] > 100) | (X_outliers['length_time_hours'] >= 3)]
    X.loc[X_outliers.index, 'length_time_hours'] = np.NaN

    # Case 4: Regular distance, regular time
    X_outliers = X[(X['trip_distance_km'] <= upperbound_distance) & (X['length_time_hours'] <= upperbound_length_time)]
    X_outliers = X_outliers[X_outliers['velocity'] > 100]
    X.loc[X_outliers.index, 'length_time_hours'] = np.NaN

    return X

def filter_fares(X):
    upperbound_distance = calculate_upperbound(X, 'trip_distance_km')
    upperbound_length_time = calculate_upperbound(X, 'length_time_hours')
    upperbound_fare_amount = calculate_upperbound(X, 'fare_amount')

    # Filter fares below 2.50
    X_outliers = X[X['fare_amount'] < 2.50]
    X.loc[X_outliers.index, 'fare_amount'] = np.NaN

    # Filter irregular fare amounts
    X_outliers = X[(X['trip_distance_km'] <= upperbound_distance) & 
                   (X['length_time_hours'] <= upperbound_length_time) & 
                   (X['fare_amount'] > upperbound_fare_amount)]
    X.loc[X_outliers.index, 'fare_amount'] = np.NaN

    return X

def filter_passenger_count(X): 
    X.loc[X['passenger_count'] == 0, 'passenger_count'] = np.NaN
    X.loc[X['passenger_count'] > 6, 'passenger_count'] = np.NaN

    return X

def convert_int_to_object(X):
    X[['vendor_id', 'weather_code (wmo code)']] = X[['vendor_id', 'weather_code (wmo code)']].astype(object)
    return X


In [158]:
def tweak_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply a series of feature-engineering, filtering, and transformation functions to the input DataFrame.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing trip data.
    
    Returns
    -------
    pd.DataFrame
        The transformed DataFrame with new features, filtered values, and selected columns.
    """
    return (df
            .pipe(length_time_hours)       # Convert length_time to hours and remove original column
            .pipe(trip_distance_km)        # Convert trip_distance to kilometers and remove original column
            .pipe(calculate_velocity)      # Calculate velocity using trip_distance_km and length_time_hours
            .pipe(cost_per_mile)           # Calculate cost per kilometer
            .pipe(cost_per_hour)           # Calculate cost per hour
            .pipe(cost_per_pair)           # Calculate fare variation based on pair
            .pipe(fare_per_passenger)      # Calculate fare per passenger
            .pipe(estimated_fare)          # Estimate fare based on trip details
            .pipe(fare_differences)        # Calculate the difference between actual and estimated fares
            .pipe(add_datetime_column)     # Add datetime and year information
            .pipe(add_weather_info)        # Add weather information by merging with external data
            .pipe(replace_inf_nan)         # Replace infinite and NaN values
            .pipe(filter_trip_distance_km) # Filter outliers based on trip distance and time
            .pipe(filter_length_time)      # Filter outliers based on length of time and velocity
            .pipe(filter_fares)            # Filter outliers based on fare amount and trip details
            .pipe(filter_passenger_count)  # Filter invalid passenger counts
            .pipe(convert_int_to_object)   # Convert categorical columns to type int
            .pipe(drop_columns)            # Drop the specified columns
           )

In [159]:
class TweakDfTransformer(base.BaseEstimator, base.TransformerMixin): 
    def __init__(self, ycol=None): 
        self.ycol = ycol

    def transform(self, X): 
        return tweak_df(X)

    def fit(self, X, y=None): 
        return self



In [160]:
# Separate the column names based on data type
float_objects = [
    'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'length_time_hours', 'trip_distance_km',
    'velocity', 'cost_per_km', 'cost_per_hour', 'fare_amount_pair_variation', 'fare_per_passenger', 'diff_fares', 'temperature_2m (°C)', 'rain (mm)', 
    'passenger_count'
]

ordinal_encoding = ['pickup_hour', 'pickup_doy']

# Assuming 'weather_code' and 'vendor_id' are the one-hot encoded variables
one_hot_variables = ['vendor_id', 'weather_code (wmo code)']

In [161]:
df_pl_pre = pipeline.Pipeline([
    ('tweak', TweakDfTransformer()), 
    ('cat', encoding.OneHotEncoder(variables=one_hot_variables)),
    ('float', imputation.MeanMedianImputer(imputation_method='mean', variables = float_objects))
])

## Applying pre-processing pipelines 
Rather than tuning all of the parameters at once, we will use step-wise tuning. We will tune small groups of hyperparameters that act similarly, then move to the next group while preserving the values from the previous group. We're going to limit our steps to:

- Tree parameters
- Sampling parameters
- Regularisation parameters
- Learning rate


In [162]:
X, y = df_train.drop(columns=["response"]), df_train["response"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train_pre = df_pl_pre.fit_transform(X_train,y_train)
X_test_pre = df_pl_pre.transform(X_test)

scaler = StandardScaler()
X_train_pre_scaled = pd.DataFrame(scaler.fit_transform(X_train_pre), columns = X_train_pre.columns)
X_test_pre_scaled = pd.DataFrame(scaler.transform(X_test_pre), columns = X_test_pre.columns)

## Step-wise hyperarameter tuning 

In [165]:
def hyperparameter_tuning(
    space: Dict[str, Union[float, int]], 
    X_train: pd.DataFrame, 
    y_train: pd.Series,
    X_test: pd.DataFrame, 
    y_test: pd.Series,
    early_stopping_rounds: int = 50,
    n_estimators=1000,
    metric: callable = mean_absolute_error ) -> Dict[str, Any]:
    """
    Perform hyperparameter tuning for an XGBoost classifier.
    
    This function takes a dictionary of hyperparameters, training
    and test data, and an optional value for early stopping rounds,
    and returns a dictionary with the loss and model resulting from
    the tuning process. The model is trained using the training
    data and evaluated on the test data. The loss is computed as
    the negative of the accuracy score.
    
    Parameters
    ----------
    space : Dict[str, Union[float, int]]
        A dictionary of hyperparameters for the XGBoost classifier.
    X_train : pd.DataFrame
        The training data.
    y_train : pd.Series
        The training target.
    X_test : pd.DataFrame
        The test data.
    y_test : pd.Series
        The test target.
    early_stopping_rounds : int, optional
        The number of early stopping rounds to use. Default is 50.
    metric : callable
        Metric to maximize. Default is accuracy score.
        
    Returns
    -------
    Dict[str, Any]
        A dictionary with the loss and model resulting from the
        tuning process. The loss is a float, and the model is an
        XGBoost classifier.
    """
    
    # Convert integer parameters
    int_vals = ['max_depth', 'reg_alpha']
    space = {k: (int(val) if k in int_vals else val) for k, val in space.items()}
    space['early_stopping_rounds'] = early_stopping_rounds
    
    # Instantiate the model with the given hyperparameters
    model = xgb.XGBRegressor(**space, objective = "reg:absoluteerror")
    
    # Define evaluation sets
    evaluation = [(X_train_pre_scaled, y_train), (X_test_pre_scaled, y_test)]
    
    # Train the model
    model.fit(
        X_train_pre_scaled, y_train,
        eval_set=evaluation,
        verbose=False
    )
    
    # Make predictions and calculate the score
    pred = model.predict(X_test_pre_scaled)
    score = metric(y_test, pred)
    
    # Return the loss and model
    return {'loss': score, 'status': STATUS_OK, 'model': model}


In [166]:
rounds = [
    {'max_depth': hp.quniform('max_depth', 1, 8, 1), 'min_child_weight': hp.loguniform('min_child_weight', -2, 3)},
    {'subsample': hp.uniform('subsample', 0.5, 1), 'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)},
    {'reg_alpha': hp.uniform('reg_alpha', 0, 10), 'reg_lambda': hp.uniform('reg_lambda', 1, 10)},
    {'gamma': hp.loguniform('gamma', -10, 10)},
    {'learning_rate': hp.loguniform('learning_rate', -7, 0)}
]

all_trials = []
params = {}

for round in rounds:
    params.update(round)
    trials = Trials()
    best = fmin(
        fn=lambda space: hyperparameter_tuning(space, X_train_pre_scaled, y_train, X_test_pre_scaled, y_test),
        space=params,
        algo=tpe.suggest,
        max_evals=20,
        trials=trials
    )
    params.update(best)
    all_trials.append(trials)

100%|██████████| 20/20 [02:47<00:00,  8.37s/trial, best loss: 0.5695556905128237]


In [171]:
final_model = xgb.XGBRegressor(
    max_depth=int(params['max_depth']),
    min_child_weight=params['min_child_weight'],
    subsample=params['subsample'],
    colsample_bytree=params['colsample_bytree'],
    reg_alpha=params['reg_alpha'],
    reg_lambda=params['reg_lambda'],
    gamma=params['gamma'],
    learning_rate=params['learning_rate'],
    early_stopping_rounds=50,
    n_estimators=1000,
    objective = "reg:absoluteerror"
)

final_model.fit(X_train_pre_scaled, y_train, 
               eval_set = [(X_train_pre_scaled, y_train), (X_test_pre_scaled, y_test)])
y_pred = final_model.predict(X_test_pre_scaled)
mae = mean_absolute_error(y_test, y_pred)
print("Test MAE:", mae)

[0]	validation_0-mae:1.04871	validation_1-mae:1.05185
[1]	validation_0-mae:0.84132	validation_1-mae:0.84301
[2]	validation_0-mae:0.72941	validation_1-mae:0.73033
[3]	validation_0-mae:0.67255	validation_1-mae:0.67313
[4]	validation_0-mae:0.63862	validation_1-mae:0.63891
[5]	validation_0-mae:0.61709	validation_1-mae:0.61737
[6]	validation_0-mae:0.60571	validation_1-mae:0.60589
[7]	validation_0-mae:0.59835	validation_1-mae:0.59851
[8]	validation_0-mae:0.59426	validation_1-mae:0.59408
[9]	validation_0-mae:0.58961	validation_1-mae:0.58956
[10]	validation_0-mae:0.58767	validation_1-mae:0.58770
[11]	validation_0-mae:0.58671	validation_1-mae:0.58676
[12]	validation_0-mae:0.58413	validation_1-mae:0.58434
[13]	validation_0-mae:0.58292	validation_1-mae:0.58332
[14]	validation_0-mae:0.58175	validation_1-mae:0.58217
[15]	validation_0-mae:0.58131	validation_1-mae:0.58172
[16]	validation_0-mae:0.58044	validation_1-mae:0.58091
[17]	validation_0-mae:0.57948	validation_1-mae:0.58006
[18]	validation_0-ma